In [5]:
import random
import dspy
from typing import Literal
from dspy.datasets import DataLoader
from datasets import load_dataset
import os
 # Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
trainset = [
    dspy.Example(x, hint=CLASSES[x.label], label=CLASSES[x.label]).with_inputs("text", "hint")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:2000]
]
random.Random(0).shuffle(trainset)

In [7]:
dspy.settings.experimental = True

# Set up the OpenAI API key and instantiate the GPT-4o model
api_key = os.getenv("OPENAI_API_KEY")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
lm = dspy.LM("openai/gpt-4o-mini", api_key=api_key)
dspy.configure(lm=lm)

# Define the DSPy module for classification. It will use the hint at training time, if available.
signature = dspy.Signature("text -> label").with_updated_fields('label', type_=Literal[tuple(CLASSES)])
classify = dspy.ChainOfThoughtWithHint(signature)

# Optimize via BootstrapFinetune.
optimizer = dspy.BootstrapFinetune(metric=(lambda x, y, trace=None: x.label == y.label), num_threads=24)
optimized = optimizer.compile(classify, trainset=trainset)

optimized(text="What does a pending cash withdrawal mean?")

[BootstrapFinetune] Preparing the student and teacher programs...
[BootstrapFinetune] Bootstrapping data...
Average Metric: 1981.00 / 2000 (99.0%): 100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [01:46<00:00, 18.85it/s]

2025/01/11 17:29:17 INFO dspy.evaluate.evaluate: Average Metric: 1981 / 2000 (99.0%)



[BootstrapFinetune] Preparing the train data...
[BootstrapFinetune] Collected data for 2000 examples
[BootstrapFinetune] After filtering with the metric, 1981 examples remain
[BootstrapFinetune] Using 1981 data points for fine-tuning the model: openai/gpt-4o-mini
[BootstrapFinetune] Starting LM fine-tuning...
[BootstrapFinetune] 1 fine-tuning job(s) to start
[BootstrapFinetune] Starting 1 fine-tuning job(s)...
[OpenAI Provider] Validating the data format
[OpenAI Provider] Saving the data to a file
[OpenAI Provider] Data saved to /home/bianders/.dspy_cache/finetune/097c3059d3432f5b.jsonl
[OpenAI Provider] Uploading the data to the provider
[OpenAI Provider] Starting remote training


2025/01/11 17:29:22 ERROR dspy.clients.lm: Error code: 400 - {'error': {'message': 'Model openai/gpt-4o-mini is not available for fine-tuning or does not exist.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}


[BootstrapFinetune] Job 1/1 is done
[BootstrapFinetune] Updating the student program with the fine-tuned LMs...
[BootstrapFinetune] BootstrapFinetune has finished compiling the student program


AttributeError: 'BadRequestError' object has no attribute 'kwargs'